<a href="https://colab.research.google.com/github/KoukiNishihara/AccessLogAnalyzer/blob/master/AccessLogAnalyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# git clone していない場合の実行





In [3]:
!git clone https://github.com/KoukiNishihara/AccessLogAnalyzer.git

fatal: destination path 'AccessLogAnalyzer' already exists and is not an empty directory.


In [4]:
%cd AccessLogAnalyzer/

/content/gdrive/My Drive/就活/fixpoint/AccessLogAnalyzer




# git clone 済みの場合の実行




In [5]:
!pip3 install apache-log-parser

In [0]:
import datetime
import apache_log_parser

In [0]:
#ファイル読み込み
def FileRead(filenames=['access_log'],host_dict={},time_list=[],time_range=[]):
    file_num=0
    if(len(time_range)==0 or len(time_range)==2):

        if(len(time_range)==2):
            start=time_range[0]
            end=time_range[1]
            # start=datetime.datetime.strptime(time_range[0], "%Y/%m/%d")
            # end=datetime.datetime.strptime(time_range[1], "%Y/%m/%d")
            flag=True

        for name in filenames:
            file_num+=1

            with open(name,'r') as fh:
                line =fh.readline()

                while line:
                    log_line_data = line_parser(line)
                    date=log_line_data["time_received_datetimeobj"].strftime("%Y/%m/%d")
                    if((len(time_range)==0) or (start<=date and end>=date)):
                        CountHost(log_line_data,host_dict)
                        CountPerHour(log_line_data,time_list)
                    line =fh.readline()
    return file_num

In [0]:
#　hostのアクセス数をカウント
def CountHost(log_line_data,host_dict):

    host_name=log_line_data["remote_host"]

    if host_name not in host_dict.keys():
        i=1
        update_data = {host_name:i}
        host_dict.update(update_data)

    else:
        host_dict[host_name]+=1

    return host_dict

In [0]:
# 単位時間あたりのカウント
def CountPerHour(log_line_data,list):

    time_log=log_line_data['time_received_datetimeobj'].hour
    time_list[time_log]+=1

    return host_dict

* メインプログラム


In [13]:
filename=['access_log']
host_dict={}
time_list=[]

time_range=input('ログを収集する期間を指定してください.指定がない場合はEnterをおしてください\n例）2000/11/06 2017/11/07\n').split()

for i in range(0,24):
     time_list.append(0)

line_parser = apache_log_parser.make_parser("%h %l %u %t \"%r\" %>s %b \"%{Referer}i\" \"%{User-Agent}i\"")
file_num=FileRead(filename,host_dict,time_list,time_range)
print()

# while filename:
filename=input('"access_log"以外に解析したいファイルがあればファイル名を入力してください．Enterを押すとプログラムが終了します.\n例）ファイル名\n例）ファイル名1　ファイル名2　・・・　ファイル名n\n').split()

file_num+=FileRead(filename,host_dict,time_list,time_range)

ログを収集する期間を指定してください.指定がない場合はEnterをおしてください
例）2000/11/06 2017/11/07
2000/11/06 2017/11/07

"access_log"以外に解析したいファイルがあればファイル名を入力してください．Enterを押すとプログラムが終了します.
例）ファイル名
例）ファイル名1　ファイル名2　・・・　ファイル名n
other_log


* 出力

In [14]:
print("解析したファイルは",file_num,"個です")

if(len(host_dict)==0):
    print("アクセスはありませんでした")
else :
    if(len(time_range)==2):
        print("解析期間は",time_range[0],"から",time_range[1],"です.")
    elif(len(time_range)==0):
            print("解析期間は全期間です.")

print()
for k,v in sorted(host_dict.items(),key=lambda x:x[1],reverse=True):
    print("host:",k,"count:",v)
print()

i=0
for n in time_list:
    print("hour:",i,"cout:",n)
    i+=1

解析したファイルは 2 個です
解析期間は 2000/11/06 から 2017/11/07 です.

host: 10.2.3.4 count: 3
host: 10.2.3.5 count: 1
host: 10.2.3.6 count: 1

hour: 0 cout: 2
hour: 1 cout: 0
hour: 2 cout: 0
hour: 3 cout: 1
hour: 4 cout: 0
hour: 5 cout: 0
hour: 6 cout: 0
hour: 7 cout: 0
hour: 8 cout: 0
hour: 9 cout: 0
hour: 10 cout: 1
hour: 11 cout: 0
hour: 12 cout: 0
hour: 13 cout: 0
hour: 14 cout: 0
hour: 15 cout: 0
hour: 16 cout: 0
hour: 17 cout: 0
hour: 18 cout: 0
hour: 19 cout: 0
hour: 20 cout: 1
hour: 21 cout: 0
hour: 22 cout: 0
hour: 23 cout: 0


このプログラムはread()やrandlines()を用いず，randline()を用いてテキストの読み込みを行っている．read()やrandlines()はテキストファイルを丸ごと読み込むのに対し，randline()は1行づつ読み取っている．これにより，変数に格納する文字列を減らし，メモリの使用量を減らしている．